In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

CATALOG_URL = "http://server:8080/catalog"
MANAGEMENT_URL = "http://server:8080/management"
DEMO_WAREHOUSE = "demo"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.5.2"
config = {
    "spark.sql.catalog.demo-catalog": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.demo-catalog.type": "rest",
    "spark.sql.catalog.demo-catalog.uri": CATALOG_URL,
    "spark.sql.catalog.demo-catalog.warehouse": DEMO_WAREHOUSE,
    "spark.sql.catalog.demo-catalog.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "demo-catalog",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}"
}
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

## Read and Write Tables

In [2]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS spark_demo")

DataFrame[]

In [3]:
spark.sql("SHOW NAMESPACES").toPandas()

,namespace
0,spark_demo


In [4]:
data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [5]:
sdf.writeTo("spark_demo.my_table").createOrReplace()

In [6]:
spark.sql("SELECT * FROM spark_demo.my_table").toPandas()

,id,strings,floats
0,1,a-string,2.2


In [7]:
spark.sql("DESCRIBE EXTENDED spark_demo.my_table").toPandas().iloc[17].data_type

'[current-snapshot-id=4537548533949952309,format=iceberg/parquet,format-version=2]'

In [16]:
spark.sql("create view spark_demo.myview4 as select id, strings from spark_demo.my_table")

DataFrame[]

In [17]:
spark.sql("select * from  spark_demo.myview4").toPandas()

,id,strings
0,1,a-string


In [11]:
spark.sparkContext.setLogLevel("debug")

In [6]:
spark.sql("CREATE OR REPLACE VIEW spark_demo.myview4 (id COMMENT 'id of thing') as select id from spark_demo.my_table")

Py4JJavaError: An error occurred while calling o44.sql.
: org.apache.iceberg.exceptions.AlreadyExistsException: Table or View with same name already exists in Namespace
	at org.apache.iceberg.rest.ErrorHandlers$ViewErrorHandler.accept(ErrorHandlers.java:158)
	at org.apache.iceberg.rest.ErrorHandlers$ViewErrorHandler.accept(ErrorHandlers.java:145)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:358)
	at org.apache.iceberg.rest.RESTClient.post(RESTClient.java:112)
	at org.apache.iceberg.rest.RESTSessionCatalog$RESTViewBuilder.create(RESTSessionCatalog.java:1205)
	at org.apache.iceberg.rest.RESTSessionCatalog$RESTViewBuilder.createOrReplace(RESTSessionCatalog.java:1225)
	at org.apache.iceberg.spark.SparkCatalog.replaceView(SparkCatalog.java:642)
	at org.apache.spark.sql.e

In [17]:
spark.sql("DROP view spark_demo.myview")

DataFrame[]

In [19]:
spark.sql("DROP table spark_demo.my_table")

DataFrame[]

In [16]:
spark.sql("select * from  spark_demo.myview3").toPandas()

,id,strings
0,1,a-string
